### Developing a real stratergy to invest in the stock market

When developing our stratergy, the goal will be to acquire good and useful information about each stock. We then need to check which parameters play a role in explaining variation in the data and which simply contribute noise to the system. Once we have generated and calculated the financial indicators and then optimised the tuning of these params using something like simulated annealing to search the complex solution space we can test and compare our stratergy against the S&P500 real life data. 

Sentiment data could be useful, financial statements even better, unsure how to extract, useful information from these datasets.

Looking for monthly readjustments for our stock portfolio. Further investigation should be carried 
on arbitrage. 

In [19]:
import yfinance as yf
import pandas as pd
import numpy as np
import os

In [20]:
tickers = ["AAPL", "MSFT", "GOOGL"]
start_date = "2015-01-01"
end_date = "2023-12-31"

In [21]:
def download_stock_data(tickers, start_date, end_date):
    stock_data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')
    return stock_data

data = download_stock_data(tickers, start_date, end_date)

# data_folder = "C:/Users/codyr/Desktop/Coding_/Quant_Trading_Basics"

# data = pd.read_csv("C:/Users/codyr/Desktop/Coding_/Quant_Trading_Basics/AAPL_stock_data.csv")

data.head()

# data.to_csv('AAPL_stock_data.csv', index=True)

[*********************100%***********************]  3 of 3 completed


Ticker           MSFT                                                        \
Price            Open       High        Low      Close  Adj Close    Volume   
Date                                                                          
2015-01-02  46.660000  47.419998  46.540001  46.759998  40.232845  27913900   
2015-01-05  46.369999  46.730000  46.250000  46.330002  39.862869  39673900   
2015-01-06  46.380001  46.750000  45.540001  45.650002  39.277802  36447900   
2015-01-07  45.980000  46.459999  45.490002  46.230000  39.776836  29114100   
2015-01-08  46.750000  47.750000  46.720001  47.590000  40.946995  29645200   

Ticker           AAPL                                                         \
Price            Open       High        Low      Close  Adj Close     Volume   
Date                                                                           
2015-01-02  27.847500  27.860001  26.837500  27.332500  24.347176  212818400   
2015-01-05  27.072500  27.162500  26.352501  26.562500  23.661270  257142000   
2015-01-06  26.635000  26.857500  26.157499  26.565001  23.663500  263188400   
2015-01-07  26.799999  27.049999  26.674999  26.937500  23.995316  160423600   
2015-01-08  27.307501  28.037500  27.174999  27.972500  24.917271  237458000   

Ticker          GOOGL                                                        
Price            Open       High        Low      Close  Adj Close    Volume  
Date                                                                         
2015-01-02  26.629999  26.790001  26.393999  26.477501  26.381865  26480000  
2015-01-05  26.357500  26.399500  25.887501  25.973000  25.879185  41182000  
2015-01-06  26.025000  26.060499  25.277500  25.332001  25.240503  54456000  
2015-01-07  25.547501  25.574499  25.182501  25.257500  25.166271  46918000  
2015-01-08  25.075500  25.375000  24.750999  25.345501  25.253954  73054000

In [22]:
multi_index_data = (
    data.stack(level=0, future_stack=True)
    .swaplevel()
    .sort_index()
)

multi_index_data

Price                    Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2015-01-02   27.847500   27.860001   26.837500   27.332500   24.347176   
       2015-01-05   27.072500   27.162500   26.352501   26.562500   23.661270   
       2015-01-06   26.635000   26.857500   26.157499   26.565001   23.663500   
       2015-01-07   26.799999   27.049999   26.674999   26.937500   23.995316   
       2015-01-08   27.307501   28.037500   27.174999   27.972500   24.917271   
...                       ...         ...         ...         ...         ...   
MSFT   2023-12-22  373.679993  375.179993  372.709991  374.579987  372.543945   
       2023-12-26  375.000000  376.940002  373.500000  374.660004  372.623505   
       2023-12-27  373.690002  375.059998  372.809998  374.070007  372.036713   
       2023-12-28  375.369995  376.459991  374.160004  375.279999  373.240112   
       2023-12-29  376.000000  377.160004  373.480011  376.040009  373.995972   

Price                 Volume  
Ticker Date                   
AAPL   2015-01-02  212818400  
       2015-01-05  257142000  
       2015-01-06  263188400  
       2015-01-07  160423600  
       2015-01-08  237458000  
...                      ...  
MSFT   2023-12-22   17091100  
       2023-12-26   12673100  
       2023-12-27   14905400  
       2023-12-28   14327000  
       2023-12-29   18723000  

[6792 rows x 6 columns]

In [63]:
### Define and calcuate RSI 
def calculate_rsi(data, column='Adj Close', window=14):
    """
    Calculate the RSI for each stock in a multi-index df
    """
    def rsi_per_stock(data):
    
        if not isinstance(data, pd.DataFrame):
            raise ValueError("Input data must be a pandas DataFrame")
    
        if column not in data.columns:
            raise ValueError(f"Column '{column}' not found in the DataFrame")
    
        # Calculate daily price changes (delta)
        delta = data['Adj Close'].diff()
    
        # Separate gains and losses
        gains = delta.clip(lower=0)
        losses = -delta.clip(upper=0)
        
        avg_gain = gains.rolling(window=window, min_periods=window).mean()
        avg_loss = losses.rolling(window=window, min_periods=window).mean()
    
        rs = avg_gain/avg_loss
        data['RSI'] = 100 - (100/ (1+rs)) # Ratio of avg gains/ avg losses during the window -> normalised to be 0 < rsi < 1
        
        return data
        
    return data.groupby(level='Ticker', group_keys=False).apply(rsi_per_stock)

def calculate_rsi_ewma(data, column="Adj Close", window=14):

    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    if column not in data.columns:
        raise ValueError(f"Column '{column}' not found in the DataFrame")

    def rsi_ewma_per_stock(data):
        # Calculate daily price changes (delta)
        delta = data[column].diff()
    
        # Separate gains and losses
        gains = delta.clip(lower=0)
        losses = -delta.clip(upper=0)
        
        avg_gain = gains.ewm(span=window, adjust=False).mean()
        avg_loss = losses.ewm(span=window, adjust=False).mean()
    
        rs = avg_gain/avg_loss
        rsi = 100 - (100/ (1+rs))

        data['RSI_EWMA'] = 100 - (100/ (1+rs)) # Ratio of exponentially smoothed gains/ losses during the window -> normalised to be 0 < rsi < 1
        return data
        
    return data.groupby(level='Ticker', group_keys=False).apply(rsi_ewma_per_stock)


def calculate_bollinger_bands(data, window=20):
    """
    Calculates Bollinger bands for multi-index df
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")
    
    def calculate_bb_per_stock(data):
        bb_data = data['Adj Close']

        rolling_mean = bb_data.rolling(window=window).mean()
        rolling_std = bb_data.rolling(window=window).std()

        data[('Bollinger', 'Middle Band')] = rolling_mean
        data[('Bollinger', 'Upper Band')] = rolling_mean + (2 * rolling_std)
        data[('Bollinger', 'Lower Band')] = rolling_mean - (2 * rolling_std)

        return data

    return data.groupby(level='Ticker', group_keys=False).apply(calculate_bb_per_stock)
    

In [64]:
column = "Adj Close"

df = calculate_rsi(multi_index_data, column)

df = calculate_rsi_ewma(df, column=column, window=14)

df = calculate_bollinger_bands(df)

df.head(25)

Price                   Open       High        Low      Close  Adj Close  \
Ticker Date                                                                
AAPL   2015-01-02  27.847500  27.860001  26.837500  27.332500  24.347176   
       2015-01-05  27.072500  27.162500  26.352501  26.562500  23.661270   
       2015-01-06  26.635000  26.857500  26.157499  26.565001  23.663500   
       2015-01-07  26.799999  27.049999  26.674999  26.937500  23.995316   
       2015-01-08  27.307501  28.037500  27.174999  27.972500  24.917271   
       2015-01-09  28.167500  28.312500  27.552500  28.002501  24.943993   
       2015-01-12  28.150000  28.157499  27.200001  27.312500  24.329359   
       2015-01-13  27.857500  28.200001  27.227501  27.555000  24.545366   
       2015-01-14  27.260000  27.622499  27.125000  27.450001  24.451836   
       2015-01-15  27.500000  27.514999  26.665001  26.705000  23.788210   
       2015-01-16  26.757500  26.895000  26.299999  26.497499  23.603376   
       2015-01-20  26.959999  27.242500  26.625000  27.180000  24.211330   
       2015-01-21  27.237499  27.764999  27.067499  27.387501  24.396166   
       2015-01-22  27.565001  28.117500  27.430000  28.100000  25.030844   
       2015-01-23  28.075001  28.437500  27.882500  28.245001  25.160006   
       2015-01-26  28.434999  28.590000  28.200001  28.275000  25.186726   
       2015-01-27  28.105000  28.120001  27.257500  27.285000  24.304857   
       2015-01-28  29.407499  29.530001  28.827499  28.827499  25.678885   
       2015-01-29  29.080000  29.797501  28.889999  29.725000  26.478354   
       2015-01-30  29.600000  30.000000  29.212500  29.290001  26.090870   
       2015-02-02  29.512501  29.792500  29.020000  29.657499  26.418222   
       2015-02-03  29.625000  29.772499  29.402500  29.662500  26.422686   
       2015-02-04  29.625000  30.127501  29.577499  29.889999  26.625336   
       2015-02-05  30.004999  30.057501  29.812500  29.985001  26.815374   
       2015-02-06  30.004999  30.062500  29.612499  29.732500  26.589558   

Price                 Volume        RSI   RSI_EWMA  (Bollinger, Middle Band)  \
Ticker Date                                                                    
AAPL   2015-01-02  212818400        NaN        NaN                       NaN   
       2015-01-05  257142000        NaN   0.000000                       NaN   
       2015-01-06  263188400        NaN   0.049986                       NaN   
       2015-01-07  160423600        NaN   7.950771                       NaN   
       2015-01-08  237458000        NaN  26.561792                       NaN   
       2015-01-09  214798000        NaN  27.055025                       NaN   
       2015-01-12  198603200        NaN  22.962072                       NaN   
       2015-01-13  268367600        NaN  27.414903                       NaN   
       2015-01-14  195826400        NaN  26.645439                       NaN   
       2015-01-15  240056000        NaN  21.666776                       NaN   
       2015-01-16  314053200        NaN  20.439437                       NaN   
       2015-01-20  199599600        NaN  34.517240                       NaN   
       2015-01-21  194303600        NaN  38.344361                       NaN   
       2015-01-22  215185600        NaN  49.936910                       NaN   
       2015-01-23  185859200  57.671264  52.053757                       NaN   
       2015-01-26  222460000  66.442696  52.532846                       NaN   
       2015-01-27  382274800  55.811143  38.052982                       NaN   
       2015-01-28  585908400  62.831000  58.578650                       NaN   
       2015-01-29  337745600  62.123848  66.116100                       NaN   
       2015-01-30  334982000  58.434375  60.009240                 24.639235   
       2015-02-02  250956400  66.039686  63.312475                 24.742788   
       2015-02-03  207662800  64.899353  63.360087                 24.880859   
       2015-02-04  28059880

In [83]:
def calculate_macd(data, column="Adj Close", fast_period=12, slow_period=26, signal_period=9):
    """
    Calculate MACD, Signal Line, and MACD Histogram for multi-index df
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_macd_per_stock(data):
        macd_data = data['Adj Close']
        
        fast_ema = macd_data.ewm(span=fast_period, adjust=False).mean()
        slow_ema = macd_data.ewm(span=slow_period, adjust=False).mean()
        
        data['MACD Line'] = macd_line = fast_ema - slow_ema
        data['Signal Line'] = signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()

        # Calculate MACD histogram 
        data['Histogram'] = macd_line - signal_line

        return data

    return data.groupby(level='Ticker', group_keys=False).apply(calculate_macd_per_stock)


def calculate_atr(data, window=14):
    """
    Calculate the average true range for multi_index_df
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_atr_per_stock(data):
        # Extract relevant columns
        high = data['High']
        low = data['Low']
        close = data['Adj Close']

        # Calculate True Range (TR)
        true_range = pd.DataFrame({
            'high_low': high - low,
            'high_prev_close': (high - close.shift()).abs(),
            'low_prev_close': (low - close.shift()).abs()
        }).max(axis=1)

        # Calculate ATR using EMA
        data['ATR'] = true_range.ewm(span=window, adjust=False).mean()

        return data

    # Apply calculation per stock
    return data.groupby(level='Ticker', group_keys=False).apply(calculate_atr_per_stock)

def calculate_roc(data, column="Adj Close", periods=14):
    """
    Calculate Rate of Change (RoC) for multi-index DataFrame.
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_roc_per_stock(data):
        # Extract relevant column
        roc_data = data[column]
        
        # Calculate RoC
        data['RoC'] = ((roc_data - roc_data.shift(periods)) / roc_data.shift(periods)) * 100

        return data

    # Apply calculation per stock
    return data.groupby(level='Ticker', group_keys=False).apply(calculate_roc_per_stock)


def calculate_obv(data):
    """
    Calculate On-Balance Volume (OBV) for a multi-index DataFrame.
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_obv_per_stock(data):
        # Extract relevant columns
        close = data['Close']
        volume = data['Volume']

        # Calculate OBV
        obv = (volume * np.sign(close.diff())).fillna(0).cumsum()

        # Add OBV to the DataFrame
        data['OBV'] = obv

        return data

    # Apply calculation per stock
    return data.groupby(level='Ticker', group_keys=False).apply(calculate_obv_per_stock)

def calculate_moving_averages(data, column="Adj Close", sma_window=50, ema_window=50):
    """
    Calculate SMA and EMA for a MultiIndex DataFrame.
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_averages_per_stock(data):
        # Extract relevant column
        price_data = data[column]
        
        # Calculate SMA
        data[f"SMA_{sma_window}"] = price_data.rolling(window=sma_window).mean()
        
        # Calculate EMA
        data[f"EMA_{ema_window}"] = price_data.ewm(span=ema_window, adjust=False).mean()

        return data

    # Apply calculation per stock
    return data.groupby(level='Ticker', group_keys=False).apply(calculate_averages_per_stock)

def calculate_garman_klass_volatility(data):
    """
    Calculate Garman-Klass Volatility for a MultiIndex DataFrame.
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_gk_per_stock(data):
        # Extract relevant columns
        high = data['High']
        low = data['Low']
        close = data['Close']
        open_price = data['Open']

        # Calculate Garman-Klass Volatility
        gk_volatility = (
            0.5 * (np.log(high / low) ** 2)
            - (2 * np.log(2) - 1) * (np.log(close / open_price) ** 2)
        )

        # Add Garman-Klass Volatility to the DataFrame
        data['GK Volatility'] = np.sqrt(gk_volatility)

        return data

    # Apply calculation per stock
    return data.groupby(level='Ticker', group_keys=False).apply(calculate_gk_per_stock)

In [84]:
df = calculate_macd(df)

df = calculate_atr(df)

df = calculate_roc(df)

df = calculate_obv(df)

df = calculate_moving_averages(df)

df = calculate_garman_klass_volatility(df)

df.tail()

Price                    Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
MSFT   2023-12-22  373.679993  375.179993  372.709991  374.579987  372.543945   
       2023-12-26  375.000000  376.940002  373.500000  374.660004  372.623505   
       2023-12-27  373.690002  375.059998  372.809998  374.070007  372.036713   
       2023-12-28  375.369995  376.459991  374.160004  375.279999  373.240112   
       2023-12-29  376.000000  377.160004  373.480011  376.040009  373.995972   

Price                Volume        RSI   RSI_EWMA  (Bollinger, Middle Band)  \
Ticker Date                                                                   
MSFT   2023-12-22  17091100  56.647123  56.133532                371.498650   
       2023-12-26  12673100  52.844259  56.326088                371.302225   
       2023-12-27  14905400  57.639859  54.297716                370.873071   
       2023-12-28  14327000  56.452998  57.886391                370.695540   
       2023-12-29  18723000  52.916449  60.153960                370.552818   

Price              (Bollinger, Upper Band)  (Bollinger, Lower Band)  \
Ticker Date                                                           
MSFT   2023-12-22               379.407022               363.590277   
       2023-12-26               378.869932               363.734517   
       2023-12-27               377.064280               364.681862   
       2023-12-28               376.352884               365.038196   
       2023-12-29               375.675101               365.430536   

Price              MACD Line  Signal Line  Histogram       ATR       RoC  \
Ticker Date                                                                
MSFT   2023-12-22   2.826478     3.868913  -1.042435  5.790797  1.473696   
       2023-12-26   2.787809     3.652692  -0.864883  5.604832  0.574471   
       2023-12-27   2.678934     3.457940  -0.779007  5.182387  1.428953   
       2023-12-28   2.659101     3.298172  -0.639072  5.081172  1.167263   
       2023-12-29   2.673556     3.173249  -0.499693  4.926335  0.483646   

Price                       OBV      SMA_50      EMA_50  GK Volatility  
Ticker Date                                                             
MSFT   2023-12-22  2.290790e+09  357.744223  359.437710       0.004425  
       2023-12-26  2.303463e+09  358.690925  359.954800       0.006458  
       2023-12-27  2.288558e+09  359.528423  360.428600       0.004208  
       2023-12-28  2.302885e+09  360.401504  360.931012       0.004331  
       2023-12-29  2.321608e+09  361.328411  361.443364       0.006933

In [85]:
def calculate_rolling_beta(data, market_column, stock_column="Adj Close", window=30):
    """
    Calculate Rolling Beta for a MultiIndex DataFrame.
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_beta_per_stock(data):
        # Calculate daily stock and market returns
        stock_returns = data[stock_column].pct_change()
        market_returns = data[market_column].pct_change()

        # Rolling covariance and variance
        rolling_cov = stock_returns.rolling(window=window).cov(market_returns)
        rolling_var = market_returns.rolling(window=window).var()

        # Calculate Rolling Beta
        data['Rolling Beta'] = rolling_cov / rolling_var

        return data

    # Apply calculation per stock
    return data.groupby(level='Ticker', group_keys=False).apply(calculate_beta_per_stock)

In [124]:
# Define the market index (e.g., S&P 500)
market_ticker = "^GSPC"

# Fetch historical data
market_data = yf.download(market_ticker, start="2015-01-01", end="2023-12-31", progress=False)

### Issue is relating to the fact that the length of them is different (stock data has every date for each stock x3)

# market_data.info()
# df.info()

# df['Market Returns'] = market_data['Adj Close']['^GSPC']#.pct_change()  # Add market returns

# market_data['Adj Close']['^GSPC']

# df['Market Returns']

# Calculate Rolling Beta
# df = calculate_rolling_beta(data, market_column='Market Returns', stock_column="Adj Close", window=30)

# df.head(30)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6792 entries, ('AAPL', Timestamp('2015-01-02 00:00:00')) to ('MSFT', Timestamp('2023-12-29 00:00:00'))
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Open                      6792 non-null   float64
 1   High                      6792 non-null   float64
 2   Low                       6792 non-null   float64
 3   Close                     6792 non-null   float64
 4   Adj Close                 6792 non-null   float64
 5   Volume                    6792 non-null   int64  
 6   RSI                       6750 non-null   float64
 7   RSI_EWMA                  6789 non-null   float64
 8   (Bollinger, Middle Band)  6735 non-null   float64
 9   (Bollinger, Upper Band)   6735 non-null   float64
 10  (Bollinger, Lower Band)   6735 non-null   float64
 11  MACD Line                 6792 non-null   float64
 12  Signal Line               6792 non-null

In [116]:
market_dates = set(market_data.index)
stock_dates = set(df.index.get_level_values('Date').unique())

dates_in_market_only = market_dates - stock_dates
dates_in_stocks_only = stock_dates - market_dates

print(len(dates_in_market_only))
print(len(dates_in_stocks_only))

0
0
